# Example genespector session

In [13]:
import scanpy as sc
import genespector.app as gp

In [14]:
adata = sc.read('./genespector/data/tiny_example2.h5ad')
adata

AnnData object with n_obs × n_vars = 448 × 18247 
    obs: 'isLK', 'gate', 'n_genes', 'n_counts', 'leiden', 'leiden_noG', 'random'
    var: 'symbol', 'geneid', 'isCC', 'n_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'draw_graph', 'leiden', 'leiden_colors', 'leiden_noG_colors', 'neighbors', 'pca'
    obsm: 'X_draw_graph_fa', 'X_draw_graph_fa_fromFIONA', 'X_draw_graph_fa_noG', 'X_umap', 'X_umap_noG'
    varm: 'PCs'

Generating + running is just one call:

In [3]:
gp.make_app(adata)

 * Serving Flask app "genespector.app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


Most AnnData files should work with genespector directly, in some cases preprocessing can be useful, see below.

## Preprocessing AnnData

### Using just sparse matrices (for large datasets)

When analysing large datasets note that using scaled values (in a dense matrix) is more memory-consuming. As an example we will generate an AnnData object with the scaled data in the .X slot and the log-normalised values in the .raw.X slot

In [15]:
adata.raw = adata
sc.pp.scale(adata)

To save resources we remove the scaled values and just use log-normalised values. It's a simple case of rebuilding the adata:


In [16]:
import anndata
adata2 = anndata.AnnData(X = adata.raw.X,
                   obs = adata.obs,
                   var = adata.raw.var,
                   uns = adata.uns,
                   obsm = adata.obsm)

In [17]:
adata2.X

<448x18247 sparse matrix of type '<class 'numpy.float32'>'
	with 1274989 stored elements in Compressed Sparse Row format>

### Categorical columns in the adata.obs

Only categorical columns from the .obs column are imported by genespector. To convert selected columns to categorical run:

In [18]:
adata.obs[['leiden', 'leiden_noG']] = adata.obs[['leiden', 'leiden_noG']].astype('category')

### Choosing coordinates

In some cases we may not want all the coordinates (e.g. 100 computed PCs) to be available from the dropdown menu. Here we will delete the X_umap_noG coordinates:

In [19]:
del adata.obsm['X_umap_noG']